In [1]:
import ect_slurm_tools as ect
import PolygonECT as poly
import numpy as np
from shape_reader import ShapeReader
import MC_ECT_distance as mc

In [2]:
#TEST 1: rotation of octohedron

In [3]:
s1=ShapeReader.shape_from_file('octahedron.off')
s1.V = s1.V-np.mean(s1.V,0)
scales = [sum(tmp**2)**(0.5) for tmp in s1.V]
s1.V = s1.V/max(scales)
s1.prepare()
s1.compute_links()
s1.compute_polygons()
s1.compute_gains2()
s1.clean_gains2()
ECT1=poly.return_ECT(s1)

v = [0,0,1]
degrees = 30
s2=ShapeReader.shape_from_file('octahedron.off')
for i in range(s2.V.shape[0]):
    s2.V[i,:] = poly.rotate_axis(s2.V[i,:], v, degrees)
s2.V = s2.V-np.mean(s2.V,0)
scales = [sum(tmp**2)**(0.5) for tmp in s2.V]
s2.V = s2.V/max(scales)
s2.prepare()
s2.compute_links()
s2.compute_polygons()
s2.compute_gains2()
s2.clean_gains2()
ECT2=poly.return_ECT(s2)

In [4]:
import os
import numpy as np
from shape_reader import ShapeReader
import math
import ect_tools
import itertools
from itertools import combinations
import pandas as pd
from numpy import genfromtxt

In [7]:
def MCECT(meshname='octahedron.off', v = [0,0,1], degrees=0, n_h = 10):
    dirs = genfromtxt('directions326.csv', delimiter=',')
    ndir=dirs.shape[0]
    heights=np.linspace(-1,1,n_h)
    s1=ShapeReader.shape_from_file(meshname)
    for i in range(s1.V.shape[0]):
        s1.V[i,:] = poly.rotate_axis(s1.V[i,:], v, degrees)
    s1.V=s1.V-np.mean(s1.V,0)
    scales=[sum(tmp**2)**(0.5) for tmp in s1.V]
    s1.V=s1.V/max(scales)
    s1.prepare()
    s1.prepare_for_MC()
    matrix=np.zeros([n_h,ndir])
    for i in range(n_h):
        threshold=heights[i]
        for j in range(ndir):
            direction=dirs[j]
            matrix[i,j]=s1.compute_MC_ECT(direction,threshold)
    return matrix

In [9]:
values = [10, 20, 50, 100, 200, 300, 500]
for n in values:
    m1 = MCECT(n_h = n)
    m2 = MCECT(degrees = 30, n_h = n)
    ectsum=np.sum((m1-m2)**2)/(326*n)
    print('n=', n, 'sum=', ectsum)

n= 10 sum= 0.06257668711656442
n= 20 sum= 0.0705521472392638
n= 50 sum= 0.0792638036809816
n= 100 sum= 0.07987730061349693
n= 200 sum= 0.0801840490797546
n= 300 sum= 0.08044989775051124
n= 500 sum= 0.08031901840490797


In [10]:
nums = [500, 1000, 5000, 10000]
for n in nums:
    mcvalue = mc.ECT_distance_MC(s1, s2, num_pts = n)
    print('n=', n, 'mcect=', mcvalue)

n= 500 mcect= 0.08
n= 1000 mcect= 0.061
n= 5000 mcect= 0.0778
n= 10000 mcect= 0.0758


In [11]:
d12=ect.compute_ECT_distance_p(ECT1, ECT2)
d11=ect.compute_ECT_distance_p(ECT1, ECT1) 
d22=ect.compute_ECT_distance_p(ECT2, ECT2)
(d11+d22-2*d12)/(8*np.pi)

array([0.08018581])

In [12]:
#TEST 2: on small real meshes(rotation)

In [13]:
s1=ShapeReader.shape_from_file('U02_sas_aligned.off')
s1.V = s1.V-np.mean(s1.V,0)
scales = [sum(tmp**2)**(0.5) for tmp in s1.V]
s1.V = s1.V/max(scales)
s1.prepare()
s1.compute_links()
s1.compute_polygons()
s1.compute_gains2()
s1.clean_gains2()
ECT1=poly.return_ECT(s1)

v = [0,0,1]
degrees = 60
s2=ShapeReader.shape_from_file('U02_sas_aligned.off')
s2.V = s2.V-np.mean(s2.V,0)
scales = [sum(tmp**2)**(0.5) for tmp in s2.V]
s2.V = s2.V/max(scales)

for i in range(s2.V.shape[0]):
    s2.V[i,:] = poly.rotate_axis(s2.V[i,:], v, degrees)

s2.prepare()
s2.compute_links()
s2.compute_polygons()
s2.compute_gains2()
s2.clean_gains2()
ECT2=poly.return_ECT(s2)

In [25]:
#reference value
nums = [500, 1000, 5000, 10000]
for n in nums:
    mcval = mc.ECT_distance_MC(s1, s2, num_pts = n)
    print('n=', n, 'mcect=', mcval)

n= 500 mcect= 0.204
n= 1000 mcect= 0.182
n= 5000 mcect= 0.1858
n= 10000 mcect= 0.1738


In [15]:
values = [10, 20, 50, 100, 200, 300, 500]
for n in values:
    m1 = MCECT(meshname='U02_sas_aligned.off', n_h = n)
    m2 = MCECT(meshname='U02_sas_aligned.off', degrees = 60, n_h = n)
    ectsum=np.sum((m1-m2)**2)/(326*n)
    print('n=', n, 'sum=', ectsum)

n= 10 sum= 0.16840490797546012
n= 20 sum= 0.17684049079754602
n= 50 sum= 0.18478527607361964
n= 100 sum= 0.18745398773006136
n= 200 sum= 0.1893558282208589
n= 300 sum= 0.18947852760736197
n= 500 sum= 0.18971779141104295


In [16]:
d12=ect.compute_ECT_distance_p(ECT1, ECT2)
d11=ect.compute_ECT_distance_p(ECT1, ECT1) 
d22=ect.compute_ECT_distance_p(ECT2, ECT2)
(d11+d22-2*d12)/(8*np.pi)

array([0.19629401])

In [17]:
# TEST 3: compare two meshes

In [18]:
s1=ShapeReader.shape_from_file('U02_sas_aligned.off')
s1.V = s1.V-np.mean(s1.V,0)
scales = [sum(tmp**2)**(0.5) for tmp in s1.V]
s1.V = s1.V/max(scales)
s1.prepare()
s1.compute_links()
s1.compute_polygons()
s1.compute_gains2()
s1.clean_gains2()
ECT1=poly.return_ECT(s1)

v = [0,0,1]
degrees = 0
s2=ShapeReader.shape_from_file('x21_sas_aligned.off')
s2.V = s2.V-np.mean(s2.V,0)
scales = [sum(tmp**2)**(0.5) for tmp in s2.V]
s2.V = s2.V/max(scales)

for i in range(s2.V.shape[0]):
    s2.V[i,:] = poly.rotate_axis(s2.V[i,:], v, degrees)

s2.prepare()
s2.compute_links()
s2.compute_polygons()
s2.compute_gains2()
s2.clean_gains2()
ECT2=poly.return_ECT(s2)

In [19]:
#reference value
nums = [500, 1000, 5000, 10000]
for n in nums:
    mcval = mc.ECT_distance_MC(s1, s2, num_pts = n)
    print('n=', n, 'mcect=', mcval)

n= 500 mcect= 0.178
n= 1000 mcect= 0.161
n= 5000 mcect= 0.171
n= 10000 mcect= 0.183


In [23]:
values = [10, 20, 50, 100, 200, 300, 500]
for n in values:
    m1 = MCECT(meshname='U02_sas_aligned.off', n_h = n)
    m2 = MCECT(meshname='x21_sas_aligned.off', n_h = n)
    ectsum=np.sum((m1-m2)**2)/(326*n)
    print('n=', n, 'sum=', ectsum)

n= 10 sum= 0.14079754601226993
n= 20 sum= 0.1668711656441718
n= 50 sum= 0.1716564417177914
n= 100 sum= 0.17042944785276073
n= 200 sum= 0.1731288343558282
n= 300 sum= 0.17367075664621676
n= 500 sum= 0.17384049079754602


In [21]:
d12=ect.compute_ECT_distance_p(ECT1, ECT2)
d11=ect.compute_ECT_distance_p(ECT1, ECT1) 
d22=ect.compute_ECT_distance_p(ECT2, ECT2)
(d11+d22-2*d12)/(8*np.pi)

array([0.17434947])